<a href="https://colab.research.google.com/github/ngonzalezromo/skullbase-res/blob/main/Zeiss_Trenion_Camera_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Skullbaseresearch.com


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import dependencies
import numpy as np
from numpy import os
import cv2
from glob import glob
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

In [53]:
#Find chessboard corners
left_mask = "/content/drive/MyDrive/stereoLeft/*.JPG"
right_mask= "/content/drive/MyDrive/stereoRight/*.JPG"
chessboardSize = (9,6)
figsize = (80, 80)


In [88]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)*5 # 5 mm square size

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.

In [80]:
#Define path of pictures 
imagesLeft = glob(left_mask)
imagesRight = glob(right_mask)

In [81]:
for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    imgL = cv2.imread(imgLeft)
    imgR = cv2.imread(imgRight)
    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv2.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv2.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv2.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv2.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        img_l_corners= cv2.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        img_r_corners= cv2.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        #cv2_imshow(img_l_corners)
        #cv2_imshow(img_r_corners)



In [82]:
############## CALIBRATION #######################################################
#alpha set to zero, no black pixels are 
frameSize= (960,1080)
retL, cameraMatrixL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpointsL, frameSize, None,4, None,None,cv2.CALIB_ZERO_TANGENT_DIST)
newCameraMatrixL, roi_L = cv2.getOptimalNewCameraMatrix(cameraMatrixL, distL, (960,1080), 0, (960,1080))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpointsR, frameSize, None,4, None, None,cv2.CALIB_ZERO_TANGENT_DIST)
newCameraMatrixR, roi_R = cv2.getOptimalNewCameraMatrix(cameraMatrixR, distR, (960,1080), 0, (960,1080))

In [83]:
########## Stereo Vision Calibration #############################################

flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv2.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, (960,1080), criteria_stereo, flags)

print("Optimal camera matrix Left camera\n", newCameraMatrixL)
print("Optimal camera matris Right camera\n", newCameraMatrixR)


Optimal camera matrix Left camera
 [[4.07696387e+03 0.00000000e+00 3.68414628e+02]
 [0.00000000e+00 8.27615234e+03 6.59844896e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Optimal camera matris Right camera
 [[4.09485962e+03 0.00000000e+00 4.06613843e+02]
 [0.00000000e+00 8.35058398e+03 6.28731350e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [84]:
########## Stereo Rectification #################################################

rectifyScale= 0 #(0, CV_CALIB_ZERO_DISPARITY)
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R= cv2.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, (960,1080), rot, trans,0,(0,0))

stereoMapL = cv2.initUndistortRectifyMap(newCameraMatrixL, distL, rectL, projMatrixL,(960,1080), cv2.CV_16SC2)
stereoMapR = cv2.initUndistortRectifyMap(newCameraMatrixR, distR, rectR, projMatrixR, (960,1080), cv2.CV_16SC2)

In [85]:
print (Q)
np.save ('Q.npy',Q)


[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -2.46626215e+03]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -8.50934520e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.31336816e+03]
 [ 0.00000000e+00  0.00000000e+00  6.26649109e-02 -0.00000000e+00]]


In [86]:
# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecsL[i], tvecsL[i], newCameraMatrixL, distL)
    error = cv2.norm(imgpointsL[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print("total error: {}".format(mean_error/len(objpoints)))

total error: 0.33864423353847395


In [87]:
print(tvecsL)

[array([[ -9.86073481],
       [-14.98204996],
       [354.72594811]]), array([[ -6.83546802],
       [-16.27436919],
       [367.42634626]]), array([[ -8.3982962 ],
       [-17.24015954],
       [375.28595011]]), array([[ -8.42047642],
       [-17.27308936],
       [375.24368237]]), array([[ -4.53209388],
       [-14.79944335],
       [366.98987999]]), array([[-12.23013802],
       [-15.52236131],
       [343.51727419]]), array([[ -9.80227976],
       [-17.09100809],
       [343.53707592]]), array([[ -7.25299917],
       [-18.00056441],
       [342.95559366]]), array([[-10.58887812],
       [-14.83234291],
       [343.53430579]]), array([[ -7.01589433],
       [-14.50204177],
       [353.94868542]]), array([[ -7.24006865],
       [-12.73722142],
       [351.80983132]]), array([[ -4.7484748 ],
       [-12.03688324],
       [359.20073534]]), array([[ -5.33335452],
       [-16.84029305],
       [371.63944881]]), array([[-15.96539896],
       [-18.10501877],
       [377.02745496]]), array